In [225]:
import sys
import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np

print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("sktime version: {}".format(sktime.__version__))
print("xgboost version: {}".format(xgb.__version__))
print("seaborn version: {}".format(sns.__version__))
print("scikit-learn version: {}".format(skl.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.8.16 | packaged by conda-forge | (default, Feb  1 2023, 15:53:35) [MSC v.1929 64 bit (AMD64)]
pandas version: 2.0.3
numpy version: 1.24.3
matplotlib version: 3.7.1
tqdm version: 4.65.0
sktime version: 0.20.1
xgboost version: 1.7.6
seaborn version: 0.12.2
scikit-learn version: 1.2.2
------------------------------------------------------------------------------


In [252]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor

pd.set_option('display.max_columns', 30)
train = pd.read_csv('train.csv')
train.drop(['num_date_time'],axis=1,inplace=True)
test = pd.read_csv('test.csv')
test.drop(['num_date_time'],axis=1,inplace=True)
test

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%)
0,1,20220825 00,23.5,0.0,2.2,72
1,1,20220825 01,23.0,0.0,0.9,72
2,1,20220825 02,22.7,0.0,1.5,75
3,1,20220825 03,22.1,0.0,1.3,78
4,1,20220825 04,21.8,0.0,1.0,77
...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84
16796,100,20220831 20,20.7,0.0,0.4,95
16797,100,20220831 21,20.2,0.0,0.4,98
16798,100,20220831 22,20.1,0.0,1.1,97


In [253]:
#train.drop(['일조(hr)', '일사(MJ/m2)'],axis=1,inplace=True)
train

,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...
203995,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


In [254]:
# '풍속(m/s)' 및 '습도(%)' 피처에 대한 NaN 값은 이전 값으로 채워줍니다.
train['풍속(m/s)'] = train['풍속(m/s)'].fillna(method='ffill')
train['습도(%)'] = train['습도(%)'].fillna(method='ffill')

# '강수량(mm)' 피처의 NaN 값은 0으로 채워줍니다.
train['강수량(mm)'] = train['강수량(mm)'].fillna(0)

train['일조(hr)'] = train['일조(hr)'].fillna(0)
train['일사(MJ/m2)'] = train['일사(MJ/m2)'].fillna(0)

# '풍속(m/s)' 및 '습도(%)' 피처에 대한 NaN 값은 이전 값으로 채워줍니다.
test['풍속(m/s)'] = test['풍속(m/s)'].fillna(method='ffill')
test['습도(%)'] = test['습도(%)'].fillna(method='ffill')

# '강수량(mm)' 피처의 NaN 값은 0으로 채워줍니다.
test['강수량(mm)'] = test['강수량(mm)'].fillna(0)

In [255]:

## 변수들을 영문명으로 변경
cols = ['num', 'date_time', 'temp', 'rainy' ,'wind','hum' , 'sun', 'MJ' ,'power']
train.columns = cols

# 'date_time' 컬럼을 문자열로 변환
train['date_time'] = train['date_time'].astype(str)

date = pd.to_datetime(train.date_time, format='%Y%m%d %H')
train['hour'] = date.dt.hour
train['day'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.isocalendar().week


cols = ['num', 'date_time', 'temp', 'rainy' ,'wind','hum']
test.columns = cols

# 'date_time' 컬럼을 문자열로 변환
test['date_time'] = test['date_time'].astype(str)

date = pd.to_datetime(test.date_time, format='%Y%m%d %H')
test['hour'] = date.dt.hour
test['day'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.isocalendar().week
test

,num,date_time,temp,rainy,wind,hum,hour,day,month,week
0,1,20220825 00,23.5,0.0,2.2,72,0,3,8,34
1,1,20220825 01,23.0,0.0,0.9,72,1,3,8,34
2,1,20220825 02,22.7,0.0,1.5,75,2,3,8,34
3,1,20220825 03,22.1,0.0,1.3,78,3,3,8,34
4,1,20220825 04,21.8,0.0,1.0,77,4,3,8,34
...,...,...,...,...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84,19,2,8,35
16796,100,20220831 20,20.7,0.0,0.4,95,20,2,8,35
16797,100,20220831 21,20.2,0.0,0.4,98,21,2,8,35
16798,100,20220831 22,20.1,0.0,1.1,97,22,2,8,35


In [256]:
train

,num,date_time,temp,rainy,wind,hum,sun,MJ,power,hour,day,month,week
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,0,2,6,22
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,1,2,6,22
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,2,2,6,22
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,3,2,6,22
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,4,2,6,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,19,2,8,34
203996,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,20,2,8,34
203997,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,21,2,8,34
203998,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,22,2,8,34


In [257]:
# train 일조, 복사량을 강수량 이용해서 test에 hour을 groupby해서 mean값 넣기
# Calculate the mean power usage for each 'num' and 'hour' in the train set
mean_power = train.groupby(['num', 'hour'])['sun'].mean().reset_index()
mean_power.rename(columns={'power': 'sun'}, inplace=True)

# Merge this with the test set
test = pd.merge(test, mean_power,  how='left', on=['num', 'hour'])

# Calculate the mean power usage for each 'num' and 'hour' in the train set
mean_power = train.groupby(['num', 'hour'])['MJ'].mean().reset_index()
mean_power.rename(columns={'power': 'MJ'}, inplace=True)

# Merge this with the test set
test = pd.merge(test, mean_power,  how='left', on=['num', 'hour'])

test

,num,date_time,temp,rainy,wind,hum,hour,day,month,week,sun,MJ
0,1,20220825 00,23.5,0.0,2.2,72,0,3,8,34,0.000000,0.0
1,1,20220825 01,23.0,0.0,0.9,72,1,3,8,34,0.000000,0.0
2,1,20220825 02,22.7,0.0,1.5,75,2,3,8,34,0.000000,0.0
3,1,20220825 03,22.1,0.0,1.3,78,3,3,8,34,0.000000,0.0
4,1,20220825 04,21.8,0.0,1.0,77,4,3,8,34,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84,19,2,8,35,0.345882,0.0
16796,100,20220831 20,20.7,0.0,0.4,95,20,2,8,35,0.047059,0.0
16797,100,20220831 21,20.2,0.0,0.4,98,21,2,8,35,0.000000,0.0
16798,100,20220831 22,20.1,0.0,1.1,97,22,2,8,35,0.000000,0.0


In [258]:
# Group by 'num', 'day', 'week', 'month'
grouped_df = test.groupby(['num', 'day', 'week', 'month'])

# Apply function to each group
def process_group(group):
    if (group['rainy'] != 0).any():  # if there is any non-zero 'rainy' value in the group
        group['sun'] = 0  # set all 'sun' values to 0
        group['MJ'] = group['MJ'] / 10  # set all 'MJ' values to 1/10 of original
    return group

# Apply the function to each group
test = grouped_df.apply(process_group).reset_index(drop=True)
test

,num,date_time,temp,rainy,wind,hum,hour,day,month,week,sun,MJ
0,1,20220829 00,22.5,0.0,1.5,66,0,0,8,35,0.000000,0.0
1,1,20220829 01,22.3,0.0,1.3,66,1,0,8,35,0.000000,0.0
2,1,20220829 02,22.3,0.0,2.6,67,2,0,8,35,0.000000,0.0
3,1,20220829 03,22.1,0.0,1.6,66,3,0,8,35,0.000000,0.0
4,1,20220829 04,21.8,0.0,1.3,66,4,0,8,35,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220828 19,22.7,0.0,0.0,65,19,6,8,34,0.345882,0.0
16796,100,20220828 20,21.9,0.0,0.3,74,20,6,8,34,0.047059,0.0
16797,100,20220828 21,20.3,0.0,0.0,84,21,6,8,34,0.000000,0.0
16798,100,20220828 22,20.6,0.0,0.1,83,22,6,8,34,0.000000,0.0


In [259]:
train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)

test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)

train['sin_day'] = np.sin(2*np.pi*train.day/7)
train['cos_day'] = np.cos(2*np.pi*train.day/7)

test['sin_day'] = np.sin(2*np.pi*test.day/7)
test['cos_day'] = np.cos(2*np.pi*test.day/7)

### 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train.loc[('2022-08-15'<=train.date_time)&(train.date_time<'2022-08-16'), 'holiday'] = 1

test['holiday'] = test.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
test.loc[('2022-08-15'<=test.date_time)&(test.date_time<'2022-08-16'), 'holiday'] = 1

## 건물별, 요일별, 시간별 발전량 평균 넣어주기
power_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['day_hour_mean'] = train.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'power'].values[0], axis = 1)
tqdm.pandas()
test['day_hour_mean'] = test.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.day == x['day']) & (power_mean.hour == x['hour']) ,'power'].values[0], axis = 1)


## 건물별 시간별 발전량 평균 넣어주기
power_hour_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean'] = train.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)
tqdm.pandas()
test['hour_mean'] = test.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)


## 건물별 시간별 발전량 표준편차 넣어주기
tqdm.pandas()
power_hour_std = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std'] = train.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)
test['hour_std'] = test.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)


100%|██████████| 16800/16800 [00:04<00:00, 3472.34it/s]


In [260]:
train['week'] = train['week'].astype('int')
test['week'] = test['week'].astype('int')

In [261]:
#불쾌지수 
train['THI'] = 9/5*train['temp'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32
test['THI'] = 9/5*test['temp'] - 0.55*(1-test['hum']/100)*(9/5*test['hum']-26)+32

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)


#냉방도일
cdhs = np.array([])
for num in range(1,101,1):
    temp = train[train['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs
cdhs = np.array([])
for num in range(1,101,1):
    temp = test[test['num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs



In [262]:
# Define SMAPE loss function
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100

In [263]:
'''
# interpolate NA values in test dataset
def interpolate_(test_df):
    # https://dacon.io/competitions/official/235736/codeshare/2844?page=1&dtype=recent
    # 에서 제안된 방법으로
    __methods = {
        'temperature': 'quadratic',
        'windspeed':'linear',
        'humidity':'quadratic',
        'precipitation':'linear',
        'insolation': 'pad'
    }

    for col, method in __methods.items():
        test_df[col] = test_df[col].interpolate(method=method)
        if method == 'quadratic':
            test_df[col] = test_df[col].interpolate(method='linear')
'''

"\n# interpolate NA values in test dataset\ndef interpolate_(test_df):\n    # https://dacon.io/competitions/official/235736/codeshare/2844?page=1&dtype=recent\n    # 에서 제안된 방법으로\n    __methods = {\n        'temperature': 'quadratic',\n        'windspeed':'linear',\n        'humidity':'quadratic',\n        'precipitation':'linear',\n        'insolation': 'pad'\n    }\n\n    for col, method in __methods.items():\n        test_df[col] = test_df[col].interpolate(method=method)\n        if method == 'quadratic':\n            test_df[col] = test_df[col].interpolate(method='linear')\n"

In [264]:
# 같은 건물? 그루핑

df = train[['date_time', 'num', 'temp', 'wind', 'hum']]

# 'date_time'이 특정 시점인 데이터에서 'temp'와 'wind'가 같은 건물 그룹 생성
group_df = df[df['date_time'] == '20220601 00'].groupby(['temp', 'wind', 'hum']).ngroup()

# 각 'num'에 대응시키기 위해 'num'과 그룹 번호를 매핑
num_group_map = df[df['date_time'] == '20220601 00'][['num']].join(group_df.rename('group'), how='inner').set_index('num')['group'].to_dict()

# 매핑한 그룹 번호를 기존 데이터프레임에 적용
train['group'] = df['num'].map(num_group_map)
test['group'] = test['num'].map(num_group_map)
test

,num,date_time,temp,rainy,wind,hum,hour,day,month,week,sun,MJ,sin_time,cos_time,sin_day,cos_day,holiday,day_hour_mean,hour_mean,hour_std,THI,CDH,group
0,1,20220829 00,22.5,0.0,1.5,66,0,0,8,35,0.000000,0.0,0.000000,1.000000,0.000000,1.00000,0,1782.90,1706.318118,446.882767,55.1464,-3.5,14
1,1,20220829 01,22.3,0.0,1.3,66,1,0,8,35,0.000000,0.0,0.258819,0.965926,0.000000,1.00000,0,1706.68,1622.620235,439.662704,54.7864,-7.2,14
2,1,20220829 02,22.3,0.0,2.6,67,2,0,8,35,0.000000,0.0,0.500000,0.866025,0.000000,1.00000,0,1602.84,1506.971294,412.071906,54.9701,-10.9,14
3,1,20220829 03,22.1,0.0,1.6,66,3,0,8,35,0.000000,0.0,0.707107,0.707107,0.000000,1.00000,0,1542.68,1437.365647,391.205981,54.4264,-14.8,14
4,1,20220829 04,21.8,0.0,1.3,66,4,0,8,35,0.000000,0.0,0.866025,0.500000,0.000000,1.00000,0,1563.42,1447.321412,381.099697,53.8864,-19.0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220828 19,22.7,0.0,0.0,65,19,6,8,34,0.345882,0.0,-0.965926,0.258819,-0.781831,0.62349,1,1045.98,1010.462118,161.399578,55.3425,-19.6,4
16796,100,20220828 20,21.9,0.0,0.3,74,20,6,8,34,0.047059,0.0,-0.866025,0.500000,-0.781831,0.62349,1,964.28,928.125176,137.566008,56.0904,-15.4,4
16797,100,20220828 21,20.3,0.0,0.0,84,21,6,8,34,0.000000,0.0,-0.707107,0.707107,-0.781831,0.62349,1,864.00,830.032941,128.300189,57.5224,-14.6,4
16798,100,20220828 22,20.6,0.0,0.1,83,22,6,8,34,0.000000,0.0,-0.500000,0.866025,-0.781831,0.62349,1,755.50,723.100235,112.464079,57.5421,-15.6,4


In [265]:
train['tem_x_hum'] = train['temp'] * train['hum']
test['tem_x_hum'] = test['temp'] * test['hum']

# Calculate mean of 'hour' for each unique 'day' and 'num'
num_day_hour_mean = train.groupby(['num', 'day', 'hour'])['power'].mean()
num_day_hour_mean.name = 'num_day_hour_mean'

# Reset the index of our Series to have a DataFrame with 'num' and 'day' as columns
num_day_hour_mean = num_day_hour_mean.reset_index()

# Merge the calculated mean to the original dataframes
train = pd.merge(train, num_day_hour_mean,  how='left', on=['num','day','hour'])
test = pd.merge(test, num_day_hour_mean,  how='left', on=['num','day','hour'])
test

,num,date_time,temp,rainy,wind,hum,hour,day,month,week,sun,MJ,sin_time,cos_time,sin_day,cos_day,holiday,day_hour_mean,hour_mean,hour_std,THI,CDH,group,tem_x_hum,num_day_hour_mean
0,1,20220829 00,22.5,0.0,1.5,66,0,0,8,35,0.000000,0.0,0.000000,1.000000,0.000000,1.00000,0,1782.90,1706.318118,446.882767,55.1464,-3.5,14,1485.0,1782.90
1,1,20220829 01,22.3,0.0,1.3,66,1,0,8,35,0.000000,0.0,0.258819,0.965926,0.000000,1.00000,0,1706.68,1622.620235,439.662704,54.7864,-7.2,14,1471.8,1706.68
2,1,20220829 02,22.3,0.0,2.6,67,2,0,8,35,0.000000,0.0,0.500000,0.866025,0.000000,1.00000,0,1602.84,1506.971294,412.071906,54.9701,-10.9,14,1494.1,1602.84
3,1,20220829 03,22.1,0.0,1.6,66,3,0,8,35,0.000000,0.0,0.707107,0.707107,0.000000,1.00000,0,1542.68,1437.365647,391.205981,54.4264,-14.8,14,1458.6,1542.68
4,1,20220829 04,21.8,0.0,1.3,66,4,0,8,35,0.000000,0.0,0.866025,0.500000,0.000000,1.00000,0,1563.42,1447.321412,381.099697,53.8864,-19.0,14,1438.8,1563.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220828 19,22.7,0.0,0.0,65,19,6,8,34,0.345882,0.0,-0.965926,0.258819,-0.781831,0.62349,1,1045.98,1010.462118,161.399578,55.3425,-19.6,4,1475.5,1045.98
16796,100,20220828 20,21.9,0.0,0.3,74,20,6,8,34,0.047059,0.0,-0.866025,0.500000,-0.781831,0.62349,1,964.28,928.125176,137.566008,56.0904,-15.4,4,1620.6,964.28
16797,100,20220828 21,20.3,0.0,0.0,84,21,6,8,34,0.000000,0.0,-0.707107,0.707107,-0.781831,0.62349,1,864.00,830.032941,128.300189,57.5224,-14.6,4,1705.2,864.00
16798,100,20220828 22,20.6,0.0,0.1,83,22,6,8,34,0.000000,0.0,-0.500000,0.866025,-0.781831,0.62349,1,755.50,723.100235,112.464079,57.5421,-15.6,4,1709.8,755.50


In [266]:
def calculate_commute_times(df):
    # Calculate the power difference with previous hour
    df['power_diff'] = df.groupby('num')['power'].diff()

    # Calculate the hour which has the maximum positive difference within 6~10 (assuming it's the start hour)
    start_hour = df[(df['power_diff'] > 0) & (df['hour'].between(6,10))].groupby('num')['power_diff'].idxmax().reset_index()
    start_hour.columns = ['num', 'hour_idx']
    start_hour['start_hour'] = df.loc[start_hour['hour_idx'], 'hour'].values
    start_hour = start_hour.drop(columns='hour_idx')

    # Calculate the hour which has the maximum negative difference within 17~22 (assuming it's the end hour)
    end_hour = df[(df['power_diff'] < 0) & (df['hour'].between(17,22))].groupby('num')['power_diff'].idxmin().reset_index()
    end_hour.columns = ['num', 'hour_idx']
    end_hour['end_hour'] = df.loc[end_hour['hour_idx'], 'hour'].values
    end_hour = end_hour.drop(columns='hour_idx')

    # Merge the calculated start and end hours to the original dataframe
    df = df.merge(start_hour, on='num')
    df = df.merge(end_hour, on='num')

    # Drop the used column
    df = df.drop(columns='power_diff')

    return df

train = calculate_commute_times(train)
train

,num,date_time,temp,rainy,wind,hum,sun,MJ,power,hour,day,month,week,sin_time,cos_time,sin_day,cos_day,holiday,day_hour_mean,hour_mean,hour_std,THI,CDH,group,tem_x_hum,num_day_hour_mean,start_hour,end_hour
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,0,2,6,22,0.000000,1.000000,0.974928,-0.222521,0,1774.744615,1706.318118,446.882767,49.6576,-7.4,14,781.2,1774.744615,10,18
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,1,2,6,22,0.258819,0.965926,0.974928,-0.222521,0,1687.347692,1622.620235,439.662704,47.7625,-15.4,14,810.0,1687.347692,10,18
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,2,2,6,22,0.500000,0.866025,0.974928,-0.222521,0,1571.483077,1506.971294,412.071906,47.2225,-23.7,14,796.5,1571.483077,10,18
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,3,2,6,22,0.707107,0.707107,0.974928,-0.222521,0,1522.153846,1437.365647,391.205981,44.7856,-33.0,14,801.6,1522.153846,10,18
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,4,2,6,22,0.866025,0.500000,0.974928,-0.222521,0,1506.793846,1447.321412,381.099697,49.0061,-40.6,14,791.2,1506.793846,10,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,19,2,8,34,-0.965926,0.258819,0.974928,-0.222521,0,964.873846,1010.462118,161.399578,63.6624,-19.6,4,1986.6,964.873846,7,17
203996,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,20,2,8,34,-0.866025,0.500000,0.974928,-0.222521,0,882.184615,928.125176,137.566008,62.4024,-20.2,4,1926.4,882.184615,7,17
203997,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,21,2,8,34,-0.707107,0.707107,0.974928,-0.222521,0,779.095385,830.032941,128.300189,64.1976,-22.3,4,1959.6,779.095385,7,17
203998,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,22,2,8,34,-0.500000,0.866025,0.974928,-0.222521,0,663.267692,723.100235,112.464079,65.0744,-25.1,4,1974.0,663.267692,7,17


In [267]:
def calculate_commute_period(df):
    # Get the start hour and end hour for each 'num'
    start_hour = df.groupby('num')['start_hour'].first().reset_index()
    end_hour = df.groupby('num')['end_hour'].first().reset_index()

    # Define a function to apply to the DataFrame
    def is_commute_period(row):
        if row['hour'] >= start_hour.loc[row['num'] - 1, 'start_hour'] and row['hour'] <= end_hour.loc[row['num'] - 1, 'end_hour']:
            return 1
        else:
            return 0

    # Apply the function to each row
    df['commute_period'] = df.apply(is_commute_period, axis=1)

    return df

# Apply the function to the DataFrame
train = calculate_commute_period(train)
train

,num,date_time,temp,rainy,wind,hum,sun,MJ,power,hour,day,month,week,sin_time,cos_time,sin_day,cos_day,holiday,day_hour_mean,hour_mean,hour_std,THI,CDH,group,tem_x_hum,num_day_hour_mean,start_hour,end_hour,commute_period
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,0,2,6,22,0.000000,1.000000,0.974928,-0.222521,0,1774.744615,1706.318118,446.882767,49.6576,-7.4,14,781.2,1774.744615,10,18,0
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,1,2,6,22,0.258819,0.965926,0.974928,-0.222521,0,1687.347692,1622.620235,439.662704,47.7625,-15.4,14,810.0,1687.347692,10,18,0
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,2,2,6,22,0.500000,0.866025,0.974928,-0.222521,0,1571.483077,1506.971294,412.071906,47.2225,-23.7,14,796.5,1571.483077,10,18,0
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,3,2,6,22,0.707107,0.707107,0.974928,-0.222521,0,1522.153846,1437.365647,391.205981,44.7856,-33.0,14,801.6,1522.153846,10,18,0
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,4,2,6,22,0.866025,0.500000,0.974928,-0.222521,0,1506.793846,1447.321412,381.099697,49.0061,-40.6,14,791.2,1506.793846,10,18,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,19,2,8,34,-0.965926,0.258819,0.974928,-0.222521,0,964.873846,1010.462118,161.399578,63.6624,-19.6,4,1986.6,964.873846,7,17,0
203996,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,20,2,8,34,-0.866025,0.500000,0.974928,-0.222521,0,882.184615,928.125176,137.566008,62.4024,-20.2,4,1926.4,882.184615,7,17,0
203997,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,21,2,8,34,-0.707107,0.707107,0.974928,-0.222521,0,779.095385,830.032941,128.300189,64.1976,-22.3,4,1959.6,779.095385,7,17,0
203998,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,22,2,8,34,-0.500000,0.866025,0.974928,-0.222521,0,663.267692,723.100235,112.464079,65.0744,-25.1,4,1974.0,663.267692,7,17,0


In [268]:
# Calculate the commute time for each 'num' in the training set
num_commute_time_map = train.groupby('num')[['start_hour', 'end_hour']].first().to_dict('index')

# Apply the commute times to the test set
for num, times in num_commute_time_map.items():
    test.loc[test['num'] == num, 'commute_period'] = ((test['hour'] >= times['start_hour']) & (test['hour'] <= times['end_hour'])).astype(int)
train.drop(['start_hour', 'end_hour'],axis=1,inplace=True)
test

,num,date_time,temp,rainy,wind,hum,hour,day,month,week,sun,MJ,sin_time,cos_time,sin_day,cos_day,holiday,day_hour_mean,hour_mean,hour_std,THI,CDH,group,tem_x_hum,num_day_hour_mean,commute_period
0,1,20220829 00,22.5,0.0,1.5,66,0,0,8,35,0.000000,0.0,0.000000,1.000000,0.000000,1.00000,0,1782.90,1706.318118,446.882767,55.1464,-3.5,14,1485.0,1782.90,0.0
1,1,20220829 01,22.3,0.0,1.3,66,1,0,8,35,0.000000,0.0,0.258819,0.965926,0.000000,1.00000,0,1706.68,1622.620235,439.662704,54.7864,-7.2,14,1471.8,1706.68,0.0
2,1,20220829 02,22.3,0.0,2.6,67,2,0,8,35,0.000000,0.0,0.500000,0.866025,0.000000,1.00000,0,1602.84,1506.971294,412.071906,54.9701,-10.9,14,1494.1,1602.84,0.0
3,1,20220829 03,22.1,0.0,1.6,66,3,0,8,35,0.000000,0.0,0.707107,0.707107,0.000000,1.00000,0,1542.68,1437.365647,391.205981,54.4264,-14.8,14,1458.6,1542.68,0.0
4,1,20220829 04,21.8,0.0,1.3,66,4,0,8,35,0.000000,0.0,0.866025,0.500000,0.000000,1.00000,0,1563.42,1447.321412,381.099697,53.8864,-19.0,14,1438.8,1563.42,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220828 19,22.7,0.0,0.0,65,19,6,8,34,0.345882,0.0,-0.965926,0.258819,-0.781831,0.62349,1,1045.98,1010.462118,161.399578,55.3425,-19.6,4,1475.5,1045.98,0.0
16796,100,20220828 20,21.9,0.0,0.3,74,20,6,8,34,0.047059,0.0,-0.866025,0.500000,-0.781831,0.62349,1,964.28,928.125176,137.566008,56.0904,-15.4,4,1620.6,964.28,0.0
16797,100,20220828 21,20.3,0.0,0.0,84,21,6,8,34,0.000000,0.0,-0.707107,0.707107,-0.781831,0.62349,1,864.00,830.032941,128.300189,57.5224,-14.6,4,1705.2,864.00,0.0
16798,100,20220828 22,20.6,0.0,0.1,83,22,6,8,34,0.000000,0.0,-0.500000,0.866025,-0.781831,0.62349,1,755.50,723.100235,112.464079,57.5421,-15.6,4,1709.8,755.50,0.0


In [269]:
power_diff_dict = {}

for num in range(1, 101): 
    weekday_mean_power = train[(train['num'] == num) & (train['day'] < 5)]['power'].mean()
    weekend_mean_power = train[(train['num'] == num) & (train['day'] >= 5)]['power'].mean()
    # Check if the difference is more than 10%
    if abs(weekday_mean_power - weekend_mean_power) / weekday_mean_power > 0.1:
        power_diff_dict[num] = 1
    else:
        power_diff_dict[num] = 0
train['power_diff'] = train['num'].map(power_diff_dict)
test['power_diff'] = test['num'].map(power_diff_dict)

power_diff_ratio_dict = {}

for num in range(1, 101): 
    weekday_mean_power = train[(train['num'] == num) & (train['day'] < 5)]['power'].mean()
    weekend_mean_power = train[(train['num'] == num) & (train['day'] >= 5)]['power'].mean()
    # Calculate the difference ratio
    power_diff_ratio = abs(weekday_mean_power - weekend_mean_power) / weekday_mean_power
    power_diff_ratio_dict[num] = power_diff_ratio

train['power_diff_ratio'] = train['num'].map(power_diff_ratio_dict)
test['power_diff_ratio'] = test['num'].map(power_diff_ratio_dict)
train

,num,date_time,temp,rainy,wind,hum,sun,MJ,power,hour,day,month,week,sin_time,cos_time,sin_day,cos_day,holiday,day_hour_mean,hour_mean,hour_std,THI,CDH,group,tem_x_hum,num_day_hour_mean,commute_period,power_diff,power_diff_ratio
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,0,2,6,22,0.000000,1.000000,0.974928,-0.222521,0,1774.744615,1706.318118,446.882767,49.6576,-7.4,14,781.2,1774.744615,0,1,0.105766
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,1,2,6,22,0.258819,0.965926,0.974928,-0.222521,0,1687.347692,1622.620235,439.662704,47.7625,-15.4,14,810.0,1687.347692,0,1,0.105766
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,2,2,6,22,0.500000,0.866025,0.974928,-0.222521,0,1571.483077,1506.971294,412.071906,47.2225,-23.7,14,796.5,1571.483077,0,1,0.105766
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,3,2,6,22,0.707107,0.707107,0.974928,-0.222521,0,1522.153846,1437.365647,391.205981,44.7856,-33.0,14,801.6,1522.153846,0,1,0.105766
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,4,2,6,22,0.866025,0.500000,0.974928,-0.222521,0,1506.793846,1447.321412,381.099697,49.0061,-40.6,14,791.2,1506.793846,0,1,0.105766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,19,2,8,34,-0.965926,0.258819,0.974928,-0.222521,0,964.873846,1010.462118,161.399578,63.6624,-19.6,4,1986.6,964.873846,0,0,0.042777
203996,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,20,2,8,34,-0.866025,0.500000,0.974928,-0.222521,0,882.184615,928.125176,137.566008,62.4024,-20.2,4,1926.4,882.184615,0,0,0.042777
203997,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,21,2,8,34,-0.707107,0.707107,0.974928,-0.222521,0,779.095385,830.032941,128.300189,64.1976,-22.3,4,1959.6,779.095385,0,0,0.042777
203998,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,22,2,8,34,-0.500000,0.866025,0.974928,-0.222521,0,663.267692,723.100235,112.464079,65.0744,-25.1,4,1974.0,663.267692,0,0,0.042777


In [270]:
test = test.copy()
test = test.sort_values(by=['num', 'date_time'])
test = test.reset_index(drop = True)
test

,num,date_time,temp,rainy,wind,hum,hour,day,month,week,sun,MJ,sin_time,cos_time,sin_day,cos_day,holiday,day_hour_mean,hour_mean,hour_std,THI,CDH,group,tem_x_hum,num_day_hour_mean,commute_period,power_diff,power_diff_ratio
0,1,20220825 00,23.5,0.0,2.2,72,0,3,8,34,0.0,0.0,0.000000,1.000000,0.433884,-0.900969,0,1627.800000,1706.318118,446.882767,58.3456,-35.3,14,1692.0,1627.800000,0.0,1,0.105766
1,1,20220825 01,23.0,0.0,0.9,72,1,3,8,34,0.0,0.0,0.258819,0.965926,0.433884,-0.900969,0,1550.080000,1622.620235,439.662704,57.4456,-35.5,14,1656.0,1550.080000,0.0,1,0.105766
2,1,20220825 02,22.7,0.0,1.5,75,2,3,8,34,0.0,0.0,0.500000,0.866025,0.433884,-0.900969,0,1431.120000,1506.971294,412.071906,57.8725,-36.4,14,1702.5,1431.120000,0.0,1,0.105766
3,1,20220825 03,22.1,0.0,1.3,78,3,3,8,34,0.0,0.0,0.707107,0.707107,0.433884,-0.900969,0,1372.200000,1437.365647,391.205981,57.9376,-37.9,14,1723.8,1372.200000,0.0,1,0.105766
4,1,20220825 04,21.8,0.0,1.0,77,4,3,8,34,0.0,0.0,0.866025,0.500000,0.433884,-0.900969,0,1381.720000,1447.321412,381.099697,56.9961,-39.8,14,1678.6,1381.720000,0.0,1,0.105766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,20220831 19,22.5,0.0,0.9,84,19,2,8,35,0.0,0.0,-0.965926,0.258819,0.974928,-0.222521,0,964.873846,1010.462118,161.399578,61.4824,-34.5,4,1890.0,964.873846,0.0,0,0.042777
16796,100,20220831 20,20.7,0.0,0.4,95,20,2,8,35,0.0,0.0,-0.866025,0.500000,0.974928,-0.222521,0,882.184615,928.125176,137.566008,65.2725,-34.4,4,1966.5,882.184615,0.0,0,0.042777
16797,100,20220831 21,20.2,0.0,0.4,98,21,2,8,35,0.0,0.0,-0.707107,0.707107,0.974928,-0.222521,0,779.095385,830.032941,128.300189,66.7056,-35.3,4,1979.6,779.095385,0.0,0,0.042777
16798,100,20220831 22,20.1,0.0,1.1,97,22,2,8,35,0.0,0.0,-0.500000,0.866025,0.974928,-0.222521,0,663.267692,723.100235,112.464079,65.7281,-36.8,4,1949.7,663.267692,0.0,0,0.042777


In [271]:
train.loc[(train['power_diff'] == 1) & (~train['day'].isin([5, 6])), 'power_diff'] = 0
test.loc[(test['power_diff'] == 1) & (~test['day'].isin([5, 6])), 'power_diff'] = 0
train

,num,date_time,temp,rainy,wind,hum,sun,MJ,power,hour,day,month,week,sin_time,cos_time,sin_day,cos_day,holiday,day_hour_mean,hour_mean,hour_std,THI,CDH,group,tem_x_hum,num_day_hour_mean,commute_period,power_diff,power_diff_ratio
0,1,20220601 00,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,0,2,6,22,0.000000,1.000000,0.974928,-0.222521,0,1774.744615,1706.318118,446.882767,49.6576,-7.4,14,781.2,1774.744615,0,0,0.105766
1,1,20220601 01,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,1,2,6,22,0.258819,0.965926,0.974928,-0.222521,0,1687.347692,1622.620235,439.662704,47.7625,-15.4,14,810.0,1687.347692,0,0,0.105766
2,1,20220601 02,17.7,0.0,1.5,45.0,0.0,0.0,974.88,2,2,6,22,0.500000,0.866025,0.974928,-0.222521,0,1571.483077,1506.971294,412.071906,47.2225,-23.7,14,796.5,1571.483077,0,0,0.105766
3,1,20220601 03,16.7,0.0,1.4,48.0,0.0,0.0,953.76,3,2,6,22,0.707107,0.707107,0.974928,-0.222521,0,1522.153846,1437.365647,391.205981,44.7856,-33.0,14,801.6,1522.153846,0,0,0.105766
4,1,20220601 04,18.4,0.0,2.8,43.0,0.0,0.0,986.40,4,2,6,22,0.866025,0.500000,0.974928,-0.222521,0,1506.793846,1447.321412,381.099697,49.0061,-40.6,14,791.2,1506.793846,0,0,0.105766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,20220824 19,23.1,0.0,0.9,86.0,0.5,0.0,881.04,19,2,8,34,-0.965926,0.258819,0.974928,-0.222521,0,964.873846,1010.462118,161.399578,63.6624,-19.6,4,1986.6,964.873846,0,0,0.042777
203996,100,20220824 20,22.4,0.0,1.3,86.0,0.0,0.0,798.96,20,2,8,34,-0.866025,0.500000,0.974928,-0.222521,0,882.184615,928.125176,137.566008,62.4024,-20.2,4,1926.4,882.184615,0,0,0.042777
203997,100,20220824 21,21.3,0.0,1.0,92.0,0.0,0.0,825.12,21,2,8,34,-0.707107,0.707107,0.974928,-0.222521,0,779.095385,830.032941,128.300189,64.1976,-22.3,4,1959.6,779.095385,0,0,0.042777
203998,100,20220824 22,21.0,0.0,0.3,94.0,0.0,0.0,640.08,22,2,8,34,-0.500000,0.866025,0.974928,-0.222521,0,663.267692,723.100235,112.464079,65.0744,-25.1,4,1974.0,663.267692,0,0,0.042777


In [45]:
#print("실제값이 100일 때 50으로 underestimate할 때의 SMAPE : {}".format(SMAPE(100, 50)))
#print("실제값이 100일 때 150으로 overestimate할 때의 SMAPE : {}".format(SMAPE(100, 150)))

실제값이 100일 때 50으로 underestimate할 때의 SMAPE : 33.33333333333333
실제값이 100일 때 150으로 overestimate할 때의 SMAPE : 20.0


# XGB

In [168]:
preds = np.array([]) 

for i in tqdm(range(100)):
    
    y_train = train.loc[train.num == i+1, 'power']
    x_train, x_test = train.loc[train.num == i+1, ].iloc[:, 2:], test.loc[test.num == i+1, ].iloc[:,1:]
    if(i == 0):
        print(x_train)
    x_train.drop(['sun', 'MJ', 'power'],axis=1,inplace=True)
    x_test = x_test[x_train.columns]
        
    xgb = XGBRegressor(n_estimators = best_it[i], tree_method='gpu_hist', gpu_id=0, seed=990313)

    xgb.set_params(**{'objective' : weighted_mse(j)})
        
    xgb.fit(x_train, y_train)
    y_pred = xgb.predict(x_test)
    preds = np.append(preds, y_pred)


  0%|          | 0/100 [00:00<?, ?it/s]


AttributeError: 'DataFrame' object has no attribute 'num'

# AutoGluon

In [277]:
from autogluon.core.metrics import make_scorer

mape_custom = make_scorer(name='SMAPE',
                                 score_func=SMAPE,
                                 optimum=0,
                                 greater_is_better=False)

In [278]:
from autogluon.tabular import TabularDataset, TabularPredictor

preds = np.array([]) 

for i in tqdm(range(100)):
    y_train = train.loc[train.num == i+1, 'power']
    x_train, x_test = train.loc[train.num == i+1, ].iloc[:, 2:], test.loc[test.num == i+1, ].iloc[:,1:]
    if(i == 0):
        print(x_train)
        print(y_train)
        print(x_test)
    x_train.drop(['power'],axis=1,inplace=True)
    x_test = x_test[x_train.columns]
    
    train_all = pd.concat([x_train,y_train], axis = 1)
    train_data = TabularDataset(train_all)
    test_data = TabularDataset(x_test)
    predictor = TabularPredictor(label='power',  eval_metric=mape_custom).fit(train_data, presets='best_quality',  ag_args_fit={'num_gpus': 0})

    y_pred = predictor.predict(test_data)
    preds = np.append(preds, y_pred)


  0%|          | 0/100 [00:00<?, ?it/s]No path specified. Models will be saved in: "AutogluonModels\ag-20230717_101853\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230717_101853\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   37.12 GB / 511.09 GB (7.3%)
Train Data Rows:    2040
Train Data Columns: 26
Label Column: power
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5063.52, 773.76, 2653.02141, 1113.23837)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as 

      temp  rainy  wind   hum  sun    MJ    power  hour  day  month  week  \
0     18.6    0.0   0.9  42.0  0.0  0.00  1085.28     0    2      6    22   
1     18.0    0.0   1.1  45.0  0.0  0.00  1047.36     1    2      6    22   
2     17.7    0.0   1.5  45.0  0.0  0.00   974.88     2    2      6    22   
3     16.7    0.0   1.4  48.0  0.0  0.00   953.76     3    2      6    22   
4     18.4    0.0   2.8  43.0  0.0  0.00   986.40     4    2      6    22   
...    ...    ...   ...   ...  ...   ...      ...   ...  ...    ...   ...   
2035  25.5    0.0   2.9  63.0  0.1  0.19  2517.12    19    2      8    34   
2036  25.0    0.0   2.6  66.0  0.0  0.01  2208.96    20    2      8    34   
2037  24.6    0.0   1.3  67.0  0.0  0.00  1806.24    21    2      8    34   
2038  24.1    0.0   2.1  70.0  0.0  0.00  1387.20    22    2      8    34   
2039  23.8    0.0   1.4  70.0  0.0  0.00  1925.28    23    2      8    34   

      sin_time  cos_time   sin_day   cos_day  holiday  day_hour_mean  \
0  

	Useless Original Features (Count: 2): ['group', 'power_diff_ratio']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Unused Original Features (Count: 2): ['num_day_hour_mean', 'power_diff']
		These features were not used to generate any of the output features. Add a feature generator compatible with these features to utilize them.
		Features can also be unused if they carry very little information, such as being categorical but having almost entirely unique values or being duplicates of other features.
		These features do not need to be present at inference time.
		('float', []) : 1 | ['num_day_hour_mean']
		('int', [])   : 1 | ['power_diff']
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 16 | ['temp', 'rainy', 'wind', 'hum', 'sun', ...]
		('int', [])   :  6 | ['hour', 'day', 'month', 'we

In [158]:
preds = pd.Series(preds)

fig, ax = plt.subplots(60, 1, figsize=(100,200), sharex = True)
ax = ax.flatten()
for i in range(1):
    train_y = train.loc[train.num == i+1, 'power'].reset_index(drop = True)
    test_y = preds[i*168:(i+1)*168]
    ax[i].scatter(np.arange(2040) , train.loc[train.num == i+1, 'power'])
    ax[i].scatter(np.arange(2040, 2040+168) , test_y)
    ax[i].tick_params(axis='both', which='major', labelsize=6)
    ax[i].tick_params(axis='both', which='minor', labelsize=4)
#plt.savefig('./predict_xgb.png')
plt.show()

In [189]:
submission = pd.read_csv('sample_submission.csv')
submission['answer'] = preds
submission.to_csv('submit_v2.csv', index = False)

In [161]:
submission

,num_date_time,answer
0,1_20220825 00,2222.780762
1,1_20220825 01,2010.115723
2,1_20220825 02,1853.690063
3,1_20220825 03,1594.897827
4,1_20220825 04,1594.897827
...,...,...
16795,100_20220831 19,901.842957
16796,100_20220831 20,872.117371
16797,100_20220831 21,781.496887
16798,100_20220831 22,658.526367


# post processing

weighted mse와 같은 맥락에서, 과도한 underestimate를 막기 위해 예측값을 후처리했습니다.
- 예측 주로부터 직전 4주(train set 마지막 28일)의 건물별 요일별 시간대별 전력소비량의 최솟값을 구한 뒤,
- test set의 같은 건물 요일 시간대의 예측값과 비교하여 만약 1번의 최솟값보다 예측값이 작다면 최솟값으로 예측값을 대체해주었습니다.
- public score 0.01 , private score 0.08 정도의 성능 향상이 있었습니다.

In [ ]:
train_to_post = pd.read_csv('./data/train.csv', encoding = 'cp949')
cols = ['num', 'date_time', 'power', 'temp', 'wind','hum' ,'prec', 'sun', 'non_elec', 'solar']
train_to_post.columns = cols
date = pd.to_datetime(train_to_post.date_time)
train_to_post['hour'] = date.dt.hour
train_to_post['day'] = date.dt.weekday
train_to_post['month'] = date.dt.month
train_to_post['week'] = date.dt.weekofyear
train_to_post = train_to_post.loc[(('2020-08-17'>train_to_post.date_time)|(train_to_post.date_time>='2020-08-18')), ].reset_index(drop = True)

pred_clip = []
test_to_post = pd.read_csv('./data/test.csv',  encoding = 'cp949')
cols = ['num', 'date_time', 'temp', 'wind','hum' ,'prec', 'sun', 'non_elec', 'solar']
test_to_post.columns = cols
date = pd.to_datetime(test_to_post.date_time)
test_to_post['hour'] = date.dt.hour
test_to_post['day'] = date.dt.weekday
test_to_post['month'] = date.dt.month
test_to_post['week'] = date.dt.weekofyear

## submission 불러오기
df = pd.read_csv('./submission/submission_xgb_noclip.csv')
for i in range(60):
    min_data = train_to_post.loc[train_to_post.num == i+1, ].iloc[-28*24:, :] ## 건물별로 직전 28일의 데이터 불러오기
    ## 요일별, 시간대별 최솟값 계산
    min_data = pd.pivot_table(min_data, values = 'power', index = ['day', 'hour'], aggfunc = min).reset_index() 
    pred = df.answer[168*i:168*(i+1)].reset_index(drop=True) ## 168개 데이터, 즉 건물별 예측값 불러오기
    day =  test_to_post.day[168*i:168*(i+1)].reset_index(drop=True) ## 예측값 요일 불러오기
    hour = test_to_post.hour[168*i:168*(i+1)].reset_index(drop=True) ## 예측값 시간 불러오기
    df_pred = pd.concat([pred, day, hour], axis = 1)
    df_pred.columns = ['pred', 'day', 'hour']
    for j in range(len(df_pred)):
        min_power = min_data.loc[(min_data.day == df_pred.day[j])&(min_data.hour == df_pred.hour[j]), 'power'].values[0]
        if df_pred.pred[j] < min_power:
            pred_clip.append(min_power)
        else:
            pred_clip.append(df_pred.pred[j])

초록색으로 표시된 값이 원래의 예측값, 주황색이 후처리된 예측값입니다.
변동이 거의 없는 건물도 있으나, 유의미하게 바뀐 건물도 확인됩니다.

In [ ]:
pred_origin = df.answer
pred_clip = pd.Series(pred_clip)

for i in range(60):
    power = train_to_post.loc[train_to_post.num == i+1, 'power'].reset_index(drop=True)
    preds = pred_clip[i*168:(i+1)*168]
    preds_origin = pred_origin[i*168:(i+1)*168]
    preds.index = range(power.index[-1], power.index[-1]+168)
    preds_origin.index = range(power.index[-1], power.index[-1]+168)
    
    plot_series(power, preds,  preds_origin, markers = [',', ',', ','])